In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
# Specify model callbacks on training
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)

model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [32,64,96],
        "fp_length" : [96,96,96],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [96,96,48],
        'dropout_rate' : [0.1,0.1],
        'lr' : 0.001,
        'batch_size' : int(64),
        'n_epochs' : int(5)
        }


In [3]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 96, 8),
    'conv2' : hp.quniform('conv2', 48, 128, 8),
    'conv3' : hp.quniform('conv3', 64, 168, 8),
    'fp' : hp.quniform('fp', 64, 196, 8),
    'dense1' : hp.quniform('dense1',96,256,32),
    'dense2' : hp.quniform('dense2',96,256,32),
    'dense3' : hp.quniform('dense3',48,128,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.0005,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,40,5) 
}

In [4]:
target_1 = 'pi3k'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)

training_p38 = [df_p38.loc[train_val_folds_p38[0][0]],
                 df_p38.loc[train_val_folds_p38[1][0]],
                 df_p38.loc[train_val_folds_p38[2][0]],
                 df_p38.loc[train_val_folds_p38[3][0]],
                 df_p38.loc[train_val_folds_p38[4][0]],
                 df_p38.loc[train_val_folds_p38[5][0]],
                 df_p38.loc[train_test_folds_p38[0]]
                 ]
validation_p38 = [df_p38.loc[train_val_folds_p38[0][1]],
                   df_p38.loc[train_val_folds_p38[1][1]],
                   df_p38.loc[train_val_folds_p38[2][1]],
                   df_p38.loc[train_val_folds_p38[3][1]],
                   df_p38.loc[train_val_folds_p38[4][1]],
                   df_p38.loc[train_val_folds_p38[5][1]],
                   df_p38.loc[train_test_folds_p38[1]]
                   ]

In [5]:
fmin_objective = partial(objective, train_sets = training_p38, val_sets = validation_p38)

In [6]:
def run_trials():

    trials_step = 0  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 0  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [7]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 555 trials to 555 (+0) trials
100%|████████████████████████████████████████████████████████████████████████| 555/555 [00:00<?, ?trial/s, best loss=?]
Best: {'conv1': 96.0, 'conv2': 104.0, 'conv3': 120.0, 'dense1': 256.0, 'dense2': 192.0, 'dense3': 96.0, 'dropout_rate': 0.3554537312557061, 'fp': 160.0, 'lr': 0.007037117031430456, 'n_epochs': 35.0}


In [8]:
best_loss = trials.trials[0]['result']['loss']
for i in range(1,len(trials.trials)):
    if (trials.trials[i]['result']['loss'] <=  best_loss):
        best_loss = trials.trials[i]['result']['loss']
        index = i
best_params = trials.trials[index]['misc']['vals']

In [9]:
from hyperparameter_tuning_GCN import GCN_hyper
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
gcn_best = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(best_params['conv1'][0]), int(best_params['conv2'][0]), int(best_params['conv3'][0])],
        "fp_length" : [int(best_params['fp'][0]), int(best_params['fp'][0]), int(best_params['fp'][0])],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(best_params['dense1'][0]), int(best_params['dense2'][0]), int(best_params['dense3'][0])],
        'dropout_rate' : [best_params['dropout_rate'][0], best_params['dropout_rate'][0]],
        'lr' : best_params['lr'][0],
        'batch_size' : int(64),
        'n_epochs' : int(best_params['n_epochs'][0])
        }
gcn = GCN_hyper(gcn_best)

In [10]:
results_val = {}
results_train = {}
for i in range(len(training_p38)):
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(validation_p38[i])
        Y_cold = validation_p38[i].Binary
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(training_p38[i])
        Y = training_p38[i].Binary
        gcn_encoder = gcn.build_encoder()
        gcn_model = gcn.build_model(gcn_encoder)
        gcn_model.fit([X_atoms_train,X_bonds_train,X_edges_train],Y,
                    batch_size = gcn_best['batch_size'],
                    epochs = gcn_best['n_epochs'],
                    verbose = 1,
                    shuffle=True,
                    validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold],Y_cold))
        y_pred_val = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
        y_pred_train = gcn_model.predict([X_atoms_train,X_bonds_train,X_edges_train])
        if i < 6:
            results_val['Fold %s'%i] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
            results_train['Fold %s'%i] = calculate_metrics(np.array(Y),y_pred_train.squeeze())
        elif i == 6:
            results_val['Test'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
            results_train['Test'] = calculate_metrics(np.array(Y),y_pred_train.squeeze())
        


LAYER 0
LAYER 1
LAYER 2


C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2680 samples, validate on 537 samples
Epoch 1/35
2680/2680 [==============================] - 6s 2ms/step - loss: 0.6679 - acc: 0.6243 - val_loss: 0.6557 - val_acc: 0.6127
Epoch 2/35
2680/2680 [==============================] - 2s 602us/step - loss: 0.6044 - acc: 0.6563 - val_loss: 0.6266 - val_acc: 0.6704
Epoch 3/35
2680/2680 [==============================] - 2s 657us/step - loss: 0.5432 - acc: 0.7366 - val_loss: 0.6573 - val_acc: 0.6480
Epoch 4/35
2680/2680 [==============================] - 1s 554us/step - loss: 0.5417 - acc: 0.7272 - val_loss: 0.7599 - val_acc: 0.6238
Epoch 5/35
2680/2680 [==============================] - 2s 561us/step - loss: 0.5004 - acc: 0.7563 - val_loss: 0.6342 - val_acc: 0.6909
Epoch 6/35
2680/2680 [==============================] - 2s 575us/step - loss: 0.5039 - acc: 0.7612 - val_loss: 0.5663 - val_acc: 0.7244
Epoch 7/35
2680/2680 [==============================] - 2s 570us/step - loss: 0.4831 - acc: 0.7799 - val_loss: 0.6176 - val_acc: 0.6890
Epo

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2680 samples, validate on 537 samples
Epoch 1/35
2680/2680 [==============================] - 6s 2ms/step - loss: 0.6699 - acc: 0.6030 - val_loss: 0.6538 - val_acc: 0.6834
Epoch 2/35
2680/2680 [==============================] - 1s 528us/step - loss: 0.6057 - acc: 0.6377 - val_loss: 0.5249 - val_acc: 0.7523
Epoch 3/35
2680/2680 [==============================] - 1s 531us/step - loss: 0.5481 - acc: 0.7116 - val_loss: 0.5130 - val_acc: 0.7728
Epoch 4/35
2680/2680 [==============================] - 1s 533us/step - loss: 0.4941 - acc: 0.7582 - val_loss: 0.5013 - val_acc: 0.7747
Epoch 5/35
2680/2680 [==============================] - 1s 546us/step - loss: 0.4807 - acc: 0.7657 - val_loss: 0.5677 - val_acc: 0.7486
Epoch 6/35
2680/2680 [==============================] - 1s 538us/step - loss: 0.4870 - acc: 0.7560 - val_loss: 0.4769 - val_acc: 0.7821
Epoch 7/35
2680/2680 [==============================] - 1s 538us/step - loss: 0.4702 - acc: 0.7728 - val_loss: 0.5107 - val_acc: 0.7784
Epo

...

Train on 2680 samples, validate on 537 samples
Epoch 1/35
2680/2680 [==============================] - 9s 3ms/step - loss: 0.6767 - acc: 0.6201 - val_loss: 0.6534 - val_acc: 0.6369
Epoch 2/35
2680/2680 [==============================] - 2s 669us/step - loss: 0.6620 - acc: 0.6280 - val_loss: 0.6151 - val_acc: 0.6369
Epoch 3/35
2680/2680 [==============================] - 2s 649us/step - loss: 0.6067 - acc: 0.6556 - val_loss: 0.5530 - val_acc: 0.6443
Epoch 4/35
2680/2680 [==============================] - 2s 640us/step - loss: 0.5630 - acc: 0.6963 - val_loss: 0.5425 - val_acc: 0.7020
Epoch 5/35
2680/2680 [==============================] - 2s 637us/step - loss: 0.5494 - acc: 0.7194 - val_loss: 0.4785 - val_acc: 0.7709
Epoch 6/35
2680/2680 [==============================] - 2s 615us/step - loss: 0.5130 - acc: 0.7470 - val_loss: 0.5036 - val_acc: 0.7169
Epoch 7/35
2680/2680 [==============================] - 2s 608us/step - loss: 0.4935 - acc: 0.7653 - val_loss: 0.4515 - val_acc: 0.7672
Epo

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2685 samples, validate on 532 samples
Epoch 1/35
2685/2685 [==============================] - 9s 3ms/step - loss: 0.6620 - acc: 0.6439 - val_loss: 0.7185 - val_acc: 0.5188
Epoch 2/35
2685/2685 [==============================] - 2s 565us/step - loss: 0.6218 - acc: 0.6536 - val_loss: 0.6610 - val_acc: 0.5188
Epoch 3/35
2685/2685 [==============================] - 2s 566us/step - loss: 0.5624 - acc: 0.6890 - val_loss: 0.5954 - val_acc: 0.6447
Epoch 4/35
2685/2685 [==============================] - 2s 572us/step - loss: 0.5222 - acc: 0.7385 - val_loss: 0.5842 - val_acc: 0.6391
Epoch 5/35
2685/2685 [==============================] - 2s 563us/step - loss: 0.5017 - acc: 0.7479 - val_loss: 0.5372 - val_acc: 0.7162
Epoch 6/35
2685/2685 [==============================] - 2s 575us/step - loss: 0.4904 - acc: 0.7661 - val_loss: 0.5335 - val_acc: 0.7368
Epoch 7/35
2685/2685 [==============================] - 1s 552us/step - loss: 0.4724 - acc: 0.7717 - val_loss: 0.5080 - val_acc: 0.7331
Epo

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3217 samples, validate on 537 samples
Epoch 1/35
3217/3217 [==============================] - 11s 3ms/step - loss: 0.6708 - acc: 0.6298 - val_loss: 0.6382 - val_acc: 0.6704
Epoch 2/35
3217/3217 [==============================] - 2s 585us/step - loss: 0.6018 - acc: 0.6571 - val_loss: 0.6078 - val_acc: 0.7318
Epoch 3/35
3217/3217 [==============================] - 2s 561us/step - loss: 0.5511 - acc: 0.7193 - val_loss: 0.5823 - val_acc: 0.7114
Epoch 4/35
3217/3217 [==============================] - 2s 573us/step - loss: 0.5046 - acc: 0.7532 - val_loss: 0.5495 - val_acc: 0.7616
Epoch 5/35
3217/3217 [==============================] - 2s 565us/step - loss: 0.4911 - acc: 0.7513 - val_loss: 0.6272 - val_acc: 0.7374
Epoch 6/35
3217/3217 [==============================] - 2s 559us/step - loss: 0.4957 - acc: 0.7619 - val_loss: 0.5036 - val_acc: 0.7225
Epoch 7/35
3217/3217 [==============================] - 2s 567us/step - loss: 0.5056 - acc: 0.7554 - val_loss: 0.4931 - val_acc: 0.7691
Ep

In [15]:
pd.DataFrame(results_train).T

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Fold 0,0.924027,1553.0,145.0,223.0,759.0,0.843969,0.839602,0.772912,0.862687
Fold 1,0.944422,1405.0,255.0,103.0,917.0,0.904470,0.782423,0.899020,0.866418
Fold 2,0.926770,1628.0,60.0,437.0,555.0,0.865787,0.902439,0.559476,0.814552
Fold 3,0.921972,1466.0,187.0,211.0,816.0,0.870721,0.813559,0.794547,0.851493
Fold 4,0.923190,1534.0,151.0,248.0,747.0,0.847410,0.831849,0.750754,0.851119
Fold 5,0.924547,1513.0,238.0,153.0,781.0,0.866469,0.766438,0.836188,0.854376
Test,0.932794,1906.0,121.0,385.0,805.0,0.876039,0.869330,0.676471,0.842711


In [14]:
pd.DataFrame(results_val).T

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Fold 0,0.816738,290.0,39.0,97.0,111.0,0.713529,0.740000,0.533654,0.746741
Fold 1,0.862590,301.0,66.0,42.0,128.0,0.789743,0.659794,0.752941,0.798883
Fold 2,0.860597,320.0,19.0,111.0,87.0,0.758857,0.820755,0.439394,0.757914
Fold 3,0.876546,337.0,37.0,65.0,98.0,0.757467,0.725926,0.601227,0.810056
Fold 4,0.871495,310.0,32.0,82.0,113.0,0.755338,0.779310,0.579487,0.787709
Fold 5,0.867393,192.0,84.0,35.0,221.0,0.856287,0.724590,0.863281,0.776316
Test,0.846375,332.0,28.0,96.0,81.0,0.690583,0.743119,0.457627,0.769088


In [ ]:
gcn_model.save('gcn.h5')